In [1]:
import sys
sys.path.append('..')
from src.utilities import *
from src.models.train_model import train_model
from src.models.param_opt import bayes_parameter_opt_lgb

## 3. Modelling

### 3.1 Load master table

In [2]:
master = pd.read_csv(os.path.join(processed_path, 'master.csv'))

### 3.2 Train / val / test split

In [3]:
x_train = master[(master.sales.isna()==False)].drop(columns = ['region', 'brand']).copy()
x_train = x_train[x_train.month >= '2020-06']
x_train['train'] =  (x_train.month <= '2021-06').astype(float)
x_train.drop(columns = 'month', inplace = True)

### 3.3 Bayesian parameter search

In [4]:
best_params = bayes_parameter_opt_lgb(X = x_train.drop(columns = ['sales', 'train']), y = x_train.sales,
                                      init_round=100, opt_round=50, n_folds=5, random_seed=6, n_estimators=10000, 
                                      learning_rate=0.01, save_path = '').max['params']
best_params

|   iter    |  target   | baggin... | featur... | lambda_l1 | lambda_l2 | max_depth | min_ch... | min_sp... | num_le... |
-------------------------------------------------------------------------------------------------------------------------
[200]	cv_agg's rmse: 1445.64 + 98.7157
[400]	cv_agg's rmse: 1221.84 + 69.6664
[600]	cv_agg's rmse: 1174.41 + 65.3778
[800]	cv_agg's rmse: 1157.94 + 60.7203
[1000]	cv_agg's rmse: 1155.76 + 56.1196
|  1        | -1.155e+0 |  0.8227   |  0.8796   |  3.644    |  1.054    |  14.2     |  40.98    |  0.06491  |  27.44    |
[200]	cv_agg's rmse: 1413.48 + 86.8813
[400]	cv_agg's rmse: 1183.22 + 68.1401
[600]	cv_agg's rmse: 1125.43 + 62.3438
[800]	cv_agg's rmse: 1110.57 + 61.1523
[1000]	cv_agg's rmse: 1104.58 + 60.9594
[1200]	cv_agg's rmse: 1100.58 + 60.7616
[1400]	cv_agg's rmse: 1099.35 + 60.7988
[1600]	cv_agg's rmse: 1098.95 + 59.5576
[1800]	cv_agg's rmse: 1098.6 + 57.8537
|  2        | -1.099e+0 |  0.9412   |  0.2973   |  1.28     |  0.07203  |  6.283   

[600]	cv_agg's rmse: 1187.66 + 72.2075
[800]	cv_agg's rmse: 1168.03 + 68.214
[1000]	cv_agg's rmse: 1161.14 + 65.2316
[1200]	cv_agg's rmse: 1159.52 + 62.7594
[1400]	cv_agg's rmse: 1159.34 + 61.2754
|  19       | -1.159e+0 |  0.9055   |  0.681    |  4.323    |  2.808    |  8.666    |  42.31    |  0.07856  |  19.26    |
[200]	cv_agg's rmse: 1321.08 + 84.3595
[400]	cv_agg's rmse: 1106.56 + 68.9507
[600]	cv_agg's rmse: 1071.91 + 61.2979
[800]	cv_agg's rmse: 1067.58 + 58.6785
|  20       | -1.067e+0 |  0.9177   |  0.5332   |  1.214    |  0.8804   |  13.83    |  20.83    |  0.03327  |  33.94    |
[200]	cv_agg's rmse: 1357.65 + 89.7303
[400]	cv_agg's rmse: 1128.26 + 72.1163
[600]	cv_agg's rmse: 1085.11 + 66.3438
[800]	cv_agg's rmse: 1075.98 + 63.9263
[1000]	cv_agg's rmse: 1074.87 + 62.4732
|  21       | -1.075e+0 |  0.8044   |  0.3843   |  1.252    |  2.254    |  10.2     |  20.91    |  0.05518  |  33.45    |
[200]	cv_agg's rmse: 1422.28 + 95.515
[400]	cv_agg's rmse: 1206.4 + 75.0757
[600]	cv_

[1200]	cv_agg's rmse: 1155.22 + 66.1084
[1400]	cv_agg's rmse: 1152.37 + 62.996
[1600]	cv_agg's rmse: 1151.65 + 61.2707
[1800]	cv_agg's rmse: 1151.7 + 59.7358
|  37       | -1.151e+0 |  0.836    |  0.4202   |  4.421    |  2.341    |  8.233    |  46.55    |  0.08208  |  44.9     |
[200]	cv_agg's rmse: 1374.53 + 95.87
[400]	cv_agg's rmse: 1148.46 + 76.6429
[600]	cv_agg's rmse: 1104.07 + 68.9424
[800]	cv_agg's rmse: 1094.92 + 66.8796
[1000]	cv_agg's rmse: 1094.22 + 64.323
|  38       | -1.094e+0 |  0.9601   |  0.3028   |  4.629    |  0.725    |  13.32    |  23.95    |  0.01619  |  33.04    |
[200]	cv_agg's rmse: 1316 + 70.0007
[400]	cv_agg's rmse: 1105.71 + 55.9786
[600]	cv_agg's rmse: 1068.61 + 50.8168
[800]	cv_agg's rmse: 1063.86 + 48.6187
[1000]	cv_agg's rmse: 1063.73 + 47.1145
|  39       | -1.063e+0 |  0.9912   |  0.7431   |  2.13     |  0.6291   |  16.72    |  6.819    |  0.06714  |  29.3     |
[200]	cv_agg's rmse: 1479.46 + 103.174
[400]	cv_agg's rmse: 1256.86 + 75.9584
[600]	cv_agg

[1400]	cv_agg's rmse: 1180.61 + 64.7624
[1600]	cv_agg's rmse: 1179.85 + 62.1342
|  56       | -1.18e+03 |  0.9903   |  0.2271   |  0.4511   |  2.311    |  16.86    |  49.5     |  0.06808  |  29.17    |
[200]	cv_agg's rmse: 1619.94 + 121.488
[400]	cv_agg's rmse: 1361.88 + 94.0762
[600]	cv_agg's rmse: 1262.5 + 82.3164
[800]	cv_agg's rmse: 1229.95 + 76.4833
[1000]	cv_agg's rmse: 1214.16 + 73.5844
[1200]	cv_agg's rmse: 1208.57 + 71.8884
[1400]	cv_agg's rmse: 1205.43 + 68.4574
[1600]	cv_agg's rmse: 1203.48 + 66.0858
[1800]	cv_agg's rmse: 1202.87 + 63.5701
|  57       | -1.203e+0 |  0.8065   |  0.1372   |  4.79     |  2.225    |  12.39    |  46.19    |  0.04493  |  26.74    |
[200]	cv_agg's rmse: 1468.85 + 76.1329
[400]	cv_agg's rmse: 1261.73 + 55.0331
[600]	cv_agg's rmse: 1190.55 + 51.3839
[800]	cv_agg's rmse: 1166.49 + 48.2886
[1000]	cv_agg's rmse: 1154.47 + 48.7321
[1200]	cv_agg's rmse: 1147.01 + 49.8888
[1400]	cv_agg's rmse: 1141.9 + 50.3237
[1600]	cv_agg's rmse: 1139.71 + 48.7047
[1800]

[400]	cv_agg's rmse: 1147.1 + 77.5656
[600]	cv_agg's rmse: 1091.2 + 69.5024
[800]	cv_agg's rmse: 1081.1 + 68.2288
[1000]	cv_agg's rmse: 1077.93 + 67.9555
|  74       | -1.077e+0 |  0.8202   |  0.2942   |  2.23     |  2.899    |  11.1     |  17.43    |  0.06738  |  28.52    |
[200]	cv_agg's rmse: 1331.74 + 72.5707
[400]	cv_agg's rmse: 1117.17 + 56.3474
[600]	cv_agg's rmse: 1078.4 + 52.005
[800]	cv_agg's rmse: 1071.43 + 49.1483
[1000]	cv_agg's rmse: 1069.78 + 47.4663
|  75       | -1.07e+03 |  0.9389   |  0.6806   |  3.476    |  0.2589   |  16.07    |  20.21    |  0.02249  |  26.68    |
[200]	cv_agg's rmse: 1534.21 + 111.661
[400]	cv_agg's rmse: 1280.76 + 83.5078
[600]	cv_agg's rmse: 1208.13 + 75.9202
[800]	cv_agg's rmse: 1186.69 + 70.3208
[1000]	cv_agg's rmse: 1177.73 + 66.5448
[1200]	cv_agg's rmse: 1173.72 + 64.05
|  76       | -1.173e+0 |  0.8592   |  0.2136   |  2.221    |  2.876    |  13.98    |  42.93    |  0.07568  |  28.73    |
[200]	cv_agg's rmse: 1393.63 + 84.0244
[400]	cv_agg'

[200]	cv_agg's rmse: 1498.32 + 108.527
[400]	cv_agg's rmse: 1259.64 + 81.1388
[600]	cv_agg's rmse: 1189.32 + 71.9844
[800]	cv_agg's rmse: 1164.4 + 69.6552
[1000]	cv_agg's rmse: 1154.47 + 64.9266
[1200]	cv_agg's rmse: 1152.05 + 60.5083
|  93       | -1.152e+0 |  0.8325   |  0.7752   |  0.8419   |  2.367    |  12.18    |  48.18    |  0.04269  |  35.12    |
[200]	cv_agg's rmse: 1402.03 + 97.355
[400]	cv_agg's rmse: 1169.96 + 72.9128
[600]	cv_agg's rmse: 1120.53 + 63.3624
[800]	cv_agg's rmse: 1110.63 + 60.8774
[1000]	cv_agg's rmse: 1108.36 + 60.6017
[1200]	cv_agg's rmse: 1107.84 + 59.0734
[1400]	cv_agg's rmse: 1107.62 + 57.6966
|  94       | -1.107e+0 |  0.9412   |  0.5077   |  1.712    |  2.174    |  12.13    |  28.31    |  0.02024  |  27.12    |
[200]	cv_agg's rmse: 1470.83 + 104.33
[400]	cv_agg's rmse: 1245.23 + 78.4713
[600]	cv_agg's rmse: 1184.54 + 71.2993
[800]	cv_agg's rmse: 1163.49 + 68.2768
[1000]	cv_agg's rmse: 1154.39 + 66.7528
[1200]	cv_agg's rmse: 1149.73 + 65.0784
[1400]	cv_a

[600]	cv_agg's rmse: 1100.36 + 69.404
[800]	cv_agg's rmse: 1090.77 + 67.5359
[1000]	cv_agg's rmse: 1088.19 + 66.8997
|  112      | -1.088e+0 |  0.9677   |  0.2105   |  4.618    |  0.1113   |  17.79    |  5.004    |  0.05405  |  32.74    |
[200]	cv_agg's rmse: 1743.73 + 131.839
[400]	cv_agg's rmse: 1469.29 + 106.711
[600]	cv_agg's rmse: 1331.26 + 88.13
[800]	cv_agg's rmse: 1280.77 + 80.7759
[1000]	cv_agg's rmse: 1256.01 + 77.6812
[1200]	cv_agg's rmse: 1244.82 + 76.9973
[1400]	cv_agg's rmse: 1236.48 + 76.2242
[1600]	cv_agg's rmse: 1232.48 + 74.8535
[1800]	cv_agg's rmse: 1229.98 + 73.6568
|  113      | -1.23e+03 |  1.0      |  0.1      |  0.0      |  3.0      |  5.0      |  5.0      |  0.001    |  26.7     |
[200]	cv_agg's rmse: 1293.53 + 72.3661
[400]	cv_agg's rmse: 1103.44 + 62.5125
[600]	cv_agg's rmse: 1073.85 + 57.4066
[800]	cv_agg's rmse: 1070.4 + 55.1902
|  114      | -1.07e+03 |  1.0      |  0.9      |  0.0      |  0.0      |  12.51    |  5.0      |  0.001    |  33.75    |
[200]	cv

[200]	cv_agg's rmse: 1347.27 + 67.4703
[400]	cv_agg's rmse: 1122.57 + 48.7873
[600]	cv_agg's rmse: 1074.71 + 43.0428
[800]	cv_agg's rmse: 1066.35 + 40.8964
[1000]	cv_agg's rmse: 1063.92 + 39.3964
[1200]	cv_agg's rmse: 1063.05 + 38.202
|  132      | -1.063e+0 |  1.0      |  0.9      |  0.0      |  0.0      |  18.0     |  20.71    |  0.1      |  21.34    |
[200]	cv_agg's rmse: 1451.87 + 73.1303
[400]	cv_agg's rmse: 1253.08 + 55.934
[600]	cv_agg's rmse: 1188.21 + 50.0019
[800]	cv_agg's rmse: 1164.24 + 48.6779
[1000]	cv_agg's rmse: 1152.34 + 51.0131
[1200]	cv_agg's rmse: 1144.77 + 51.3481
[1400]	cv_agg's rmse: 1139.67 + 50.4512
[1600]	cv_agg's rmse: 1137.66 + 49.4989
[1800]	cv_agg's rmse: 1135.59 + 48.785
[2000]	cv_agg's rmse: 1134.46 + 47.2971
|  133      | -1.134e+0 |  1.0      |  0.9      |  5.0      |  0.0      |  5.0      |  5.0      |  0.001    |  36.62    |
[200]	cv_agg's rmse: 1324.2 + 91.0732
[400]	cv_agg's rmse: 1104.55 + 75.375
[600]	cv_agg's rmse: 1071.86 + 67.8125
[800]	cv_agg

{'bagging_fraction': 1.0,
 'feature_fraction': 0.9,
 'lambda_l1': 5.0,
 'lambda_l2': 0.0,
 'max_depth': 9.844998933631521,
 'min_child_weight': 8.630785226702193,
 'min_split_gain': 0.1,
 'num_leaves': 26.360838708888842}

### 3.4 Model training

In [4]:
##### Define best parameters found earlier
best_params = {'metric': 'rmse',
               'bagging_fraction': 1,
               'feature_fraction': 0.9,
               'lambda_l1': 5,
               'lambda_l2': 0,
               'max_depth': 9,
               'min_child_weight': 8.630785226702193,
               'min_split_gain': 0.1,
               'num_leaves': 26}

##### Train a quantile regression lgb at different alpha levels
quantile_alphas = [0.2, 0.5, 0.8]
lgb_quantiles = {}

for alpha in quantile_alphas:
    current_model = train_model(x_train, target_name = 'sales', 
                                model_type = 'lgb-quantile', 
                                quantile_alpha = alpha,
                                params = best_params, 
                                metric = 'rmse', 
                                split = 'in_sample',
                                save_path = os.path.join(models_path, 
                                                         'model2_quantile_'+ str(alpha)+'.pkl'))
    lgb_quantiles[alpha] = current_model
    
##### Visualize feature contributions
feature_contributions = pd.DataFrame({'feature': x_train.drop(columns = ['sales', 'train']).columns, 
                                      'gain': lgb_quantiles[0.5].feature_importance(importance_type = 'gain'),
                                      'split': lgb_quantiles[0.5].feature_importance(importance_type = 'split')
                                     }).sort_values('gain', ascending = False)

feature_contributions[feature_contributions.gain>0][:40]

# 0.2 - 1876.25  1789.02
# 0.5 - 1317.57  1372.7
# 0.8 -  923.42  1110.04

####################     training with  4077      ####################
####################     validating with  453      ####################
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.029529 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 31545
[LightGBM] [Info] Number of data points in the train set: 4077, number of used features: 459
[LightGBM] [Info] Start training from score 78.755775
[1]	training's rmse: 3186.01	valid_1's rmse: 2978.67
Training until validation scores don't improve for 20 rounds
[2]	training's rmse: 3163.91	valid_1's rmse: 2956
[3]	training's rmse: 3135.09	valid_1's rmse: 2927.83
[4]	training's rmse: 3093.1	valid_1's rmse: 2886.78
[5]	training's rmse: 3066.93	valid_1's rmse: 2861.35
[6]	training's rmse: 3038.63	valid_1's rmse: 2832.74
[7]	training's rmse: 3008.88	valid_1's rmse: 2803.74
[8]	training's rmse: 2980.08	valid_1's rmse: 2773.86
[9]	training's rmse: 2941.57	

[162]	training's rmse: 2068.47	valid_1's rmse: 1951.5
[163]	training's rmse: 2068.32	valid_1's rmse: 1951.43
[164]	training's rmse: 2068.35	valid_1's rmse: 1951.62
[165]	training's rmse: 2068	valid_1's rmse: 1951.17
[166]	training's rmse: 2067.81	valid_1's rmse: 1951.11
[167]	training's rmse: 2067.64	valid_1's rmse: 1950.98
[168]	training's rmse: 2067.03	valid_1's rmse: 1950.49
[169]	training's rmse: 2066.97	valid_1's rmse: 1950.45
[170]	training's rmse: 2066.03	valid_1's rmse: 1950.04
[171]	training's rmse: 2065.75	valid_1's rmse: 1949.66
[172]	training's rmse: 2065.49	valid_1's rmse: 1949.47
[173]	training's rmse: 2065.31	valid_1's rmse: 1949.24
[174]	training's rmse: 2064.43	valid_1's rmse: 1948.44
[175]	training's rmse: 2064.34	valid_1's rmse: 1948.41
[176]	training's rmse: 2055.99	valid_1's rmse: 1939.46
[177]	training's rmse: 2053.44	valid_1's rmse: 1938.65
[178]	training's rmse: 2053.24	valid_1's rmse: 1938.49
[179]	training's rmse: 2053	valid_1's rmse: 1938.29
[180]	training's 

[335]	training's rmse: 2015.17	valid_1's rmse: 1912.12
[336]	training's rmse: 2014.35	valid_1's rmse: 1911.57
[337]	training's rmse: 2014.31	valid_1's rmse: 1911.58
[338]	training's rmse: 2014.19	valid_1's rmse: 1911.51
[339]	training's rmse: 2014.03	valid_1's rmse: 1911.44
[340]	training's rmse: 2013.48	valid_1's rmse: 1911
[341]	training's rmse: 2013.46	valid_1's rmse: 1910.96
[342]	training's rmse: 2013.42	valid_1's rmse: 1910.94
[343]	training's rmse: 2013.32	valid_1's rmse: 1910.85
[344]	training's rmse: 2013.25	valid_1's rmse: 1910.79
[345]	training's rmse: 2013.1	valid_1's rmse: 1910.68
[346]	training's rmse: 2013.02	valid_1's rmse: 1910.64
[347]	training's rmse: 2012.98	valid_1's rmse: 1910.6
[348]	training's rmse: 2012.7	valid_1's rmse: 1910.45
[349]	training's rmse: 2012.63	valid_1's rmse: 1910.31
[350]	training's rmse: 2012.35	valid_1's rmse: 1909.99
[351]	training's rmse: 2012	valid_1's rmse: 1909.78
[352]	training's rmse: 2011.94	valid_1's rmse: 1909.78
[353]	training's rm

####################     training with  4077      ####################
####################     validating with  453      ####################
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.030926 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 31545
[LightGBM] [Info] Number of data points in the train set: 4077, number of used features: 459
[LightGBM] [Info] Start training from score 795.045044
[1]	training's rmse: 2797.11	valid_1's rmse: 2582.97
Training until validation scores don't improve for 20 rounds
[2]	training's rmse: 2705.37	valid_1's rmse: 2491.49
[3]	training's rmse: 2626.7	valid_1's rmse: 2417.87
[4]	training's rmse: 2557.91	valid_1's rmse: 2356.53
[5]	training's rmse: 2490.26	valid_1's rmse: 2290.28
[6]	training's rmse: 2421.7	valid_1's rmse: 2222.06
[7]	training's rmse: 2372.53	valid_1's rmse: 2178.3
[8]	training's rmse: 2320.44	valid_1's rmse: 2124.14
[9]	training's rmse: 2274.0

[162]	training's rmse: 1390	valid_1's rmse: 1414.59
[163]	training's rmse: 1389.89	valid_1's rmse: 1414.57
[164]	training's rmse: 1389.6	valid_1's rmse: 1414.5
[165]	training's rmse: 1389.42	valid_1's rmse: 1414.75
[166]	training's rmse: 1389.15	valid_1's rmse: 1415.39
[167]	training's rmse: 1389.01	valid_1's rmse: 1415.37
Early stopping, best iteration is:
[147]	training's rmse: 1393.17	valid_1's rmse: 1412.54
####################     training with  4077      ####################
####################     validating with  453      ####################
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.027492 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 31545
[LightGBM] [Info] Number of data points in the train set: 4077, number of used features: 459
[LightGBM] [Info] Start training from score 2939.709961
[1]	training's rmse: 2761.93	valid_1's rmse: 2616.75
Training until validation scores don't i

,feature,gain,split
128,sales_region_b3_6mo,6016.593522,143
8,sales_univ_brand_1mo,1174.493339,67
0,month_indicator,1006.676020,23
123,sales_univ_brand_cumstd,853.485757,24
136,sales_region_b3_cummin,309.325301,43
147,sales_region_b3market_trend_6mo,293.232199,37
1,sales_univ_b1_1mo,285.110366,5
178,region_n_intmedicine_perperson,252.497356,88
166,region_area,242.769689,77
135,sales_region_b3_cummax,240.972506,30


### 3.5 Prediction storage

In [5]:
submission = master[(master.sales.isna()) & (master.month >= '2020-07')].copy()
submission['sales'] = lgb_quantiles[0.5].predict(submission.drop(columns = ['month', 'region', 'brand', 'sales']))
submission['lower'] = lgb_quantiles[0.2].predict(submission.drop(columns = ['month', 'region', 'brand', 'sales']))
submission['upper'] = lgb_quantiles[0.8].predict(submission.drop(columns = ['month', 'region', 'brand', 'sales', 'lower']))
submission = submission[['month', 'region', 'brand', 'sales', 'lower', 'upper']]

submission.loc[submission.sales < 0, 'sales'] = 0
submission.loc[submission.lower < 0, 'lower'] = 0
submission.loc[submission.upper < 0, 'upper'] = 0
submission.head()

,month,region,brand,sales,lower,upper
2714,2020-07,region_151,brand_1,0.504487,1.057309e-20,165.524465
2715,2020-07,region_151,brand_2,0.504487,1.057309e-20,165.524465
2716,2020-07,region_152,brand_1,60.028346,1.057309e-20,444.287077
2717,2020-07,region_152,brand_2,60.028346,1.057309e-20,444.287077
2718,2020-07,region_153,brand_1,52.411337,1.057309e-20,294.961831


In [6]:
submission.to_csv(os.path.join(results_path, 'submission7_team46.csv'), index = False)

In [7]:
(submission['upper'] - submission['lower']).mean()

# Submission 2 difference - 1815.437323083508
# Submission 3 difference - 1613.6602449420075
# Submission 4 difference - 1145.0976140382802
# Submission 5 difference - 1132.1785576554485
1250.0197504629882

1250.0197504629882